In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:

class TestDataset(Dataset):
    def __init__(self, data_nums = 100000):
        self.dataset = []
        for i in range(data_nums):
            data = {}
            data["output"] = torch.randint(0,2,(1,))
            data["input"] = torch.randn(2,10)
            if data["output"] == 1:
                rand_idx = torch.randperm(10)
                data["input"],_ = torch.sort(data["input"])
                data["input"][0] = torch.tensor([data["input"][0][idx] for idx in rand_idx])
                data["input"][1] = torch.tensor([data["input"][1][idx] for idx in rand_idx])
            data["output"] = data["output"].float()
            self.dataset.append(data)
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        return self.dataset[idx]

In [3]:
class Model(nn.Module):
    def __init__(self, input_length = 10, output_length = 10):
        super(Model, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv1d(2,4,kernel_size=3,padding=1),
            nn.Conv1d(4,8,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(8,16,kernel_size=3,padding=1),
            nn.Conv1d(16,32,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(32,16,kernel_size=3,padding=1),
            nn.Conv1d(16,8,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(8,4,kernel_size=3,padding=1),
            nn.Conv1d(4,2,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Conv1d(2,1,kernel_size=3,padding=1),
            nn.Conv1d(1,1,kernel_size=3,padding=1),
            nn.Tanh(),
            nn.Linear(10,1),
            nn.Softmax(dim=0)
        )
#         self.Conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
    def forward(self, data):
        out = self.seq(data)
        return out
        

In [4]:
dataset = TestDataset(data_nums = 40960)
model = Model()

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.0001)
MSE = nn.MSELoss()
BCE = nn.BCELoss()

CS = nn.CosineSimilarity(dim = 2)

data_loader = torch.utils.data.DataLoader(dataset, batch_size = 1024, shuffle = False)
model.to(device)


Model(
  (seq): Sequential(
    (0): Conv1d(2, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(4, 8, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): Tanh()
    (3): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): Tanh()
    (6): Conv1d(32, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): Conv1d(16, 8, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): Tanh()
    (9): Conv1d(8, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (10): Conv1d(4, 2, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): Tanh()
    (12): Conv1d(2, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (13): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (14): Tanh()
    (15): Linear(in_features=10, out_features=1, bias=True)
    (16): Softmax(dim=0)
  )
)

In [6]:
from tqdm import tqdm
import numpy as np
import copy

before_param_data = {}


# for name, param in model.named_parameters():
#     if param.requires_grad:
#         before_param_data[name] = copy.deepcopy(param.data)
criterion = BCE

for epoch in range(50000):
    loss_mean = []
    for i, batch in (enumerate(data_loader)):
        input_data, GT = (batch["input"].to(device), batch["output"].to(device))
        GT = GT.unsqueeze(1)
        optimizer.zero_grad()
        predictTE = model(input_data)
        if batch == 201:
            predictTE *=500
        elif batch == 202:
            predictTE *=400
        elif batch == 203:
            predictTE *=200
        elif batch == 204:
            predictTE *=100
        elif batch == 205:
            predictTE *=50
        elif batch == 206:
            predictTE *=25
        elif batch == 207:
            predictTE *=12
        
        loss = criterion(predictTE, GT)
        loss.backward()
        optimizer.step()
        
        loss_mean.append(loss.cpu().item())
    if epoch % 20 == 19:
        print(np.mean(loss_mean))
        print(round(predictTE[0].cpu().item(),5), GT[0].cpu().item())
        print(round(predictTE[1].cpu().item(),5), GT[1].cpu().item())
        print(round(predictTE[2].cpu().item(),5), GT[2].cpu().item())
        print(round(predictTE[3].cpu().item(),5), GT[3].cpu().item())
        print("=" * 80)
    if epoch == 200:
        print("CHANGE_MODEL!!!!!!")
        criterion = MSE
        model.seq = model.seq[:16]
    elif 208>epoch >= 201:
        print(np.mean(loss_mean))
        print(round(predictTE[0].cpu().item(),5), GT[0].cpu().item())
        print(round(predictTE[1].cpu().item(),5), GT[1].cpu().item())
        print(round(predictTE[2].cpu().item(),5), GT[2].cpu().item())
        print(round(predictTE[3].cpu().item(),5), GT[3].cpu().item())
        print("=" * 80)
        

3.1650609493255617
0.00032 0.0
0.00194 1.0
0.00183 1.0
0.00178 1.0
3.1356741786003113
0.00017 0.0
0.002 1.0
0.00188 1.0
0.00185 1.0
3.124483662843704
0.0001 0.0
0.00206 1.0
0.00189 1.0
0.00185 1.0
3.11893550157547
7e-05 0.0
0.00206 1.0
0.00187 1.0
0.00183 1.0
3.115245932340622
5e-05 0.0
0.00206 1.0
0.00183 1.0
0.00182 1.0
3.1124206483364105
4e-05 0.0
0.00205 1.0
0.0018 1.0
0.0018 1.0
3.1107671439647673
4e-05 0.0
0.00206 1.0
0.00177 1.0
0.00177 1.0
3.109400045871735
3e-05 0.0
0.00206 1.0
0.00176 1.0
0.00174 1.0
3.1083071708679197
3e-05 0.0
0.00206 1.0
0.00175 1.0
0.00171 1.0
3.1074218094348907
3e-05 0.0
0.00206 1.0
0.00174 1.0
0.00167 1.0
CHANGE_MODEL!!!!!!
7.606624376773834
0.26882 0.0
1.67745 1.0
3.717 1.0
0.12918 1.0
3.6262928545475006
0.28791 0.0
-0.25909 1.0
1.92754 1.0
2.43058 1.0
1.264677394926548
0.62408 0.0
-0.4053 1.0
1.21483 1.0
1.77952 1.0
0.40134411342442033
0.58252 0.0
0.09458 1.0
0.84571 1.0
1.12628 1.0
0.21645426675677298
0.56138 0.0
0.33111 1.0
0.7464 1.0
0.91029 1.0
0.

0.012529416289180518
0.07913 0.0
1.00199 1.0
0.96406 1.0
0.94461 1.0
0.012430433998815715
0.07745 0.0
1.00168 1.0
0.96514 1.0
0.94621 1.0
0.012329554953612386
0.07569 0.0
1.00138 1.0
0.9662 1.0
0.94773 1.0
0.012226657499559224
0.07384 0.0
1.0011 1.0
0.96723 1.0
0.94916 1.0
0.012121643882710486
0.07192 0.0
1.00086 1.0
0.96824 1.0
0.95048 1.0
0.012014321400783956
0.06993 0.0
1.00067 1.0
0.96925 1.0
0.95167 1.0
0.011904280760791152
0.06791 0.0
1.00058 1.0
0.97026 1.0
0.95271 1.0
0.011790773458778858
0.06589 0.0
1.00058 1.0
0.97129 1.0
0.95357 1.0


KeyboardInterrupt: 

In [7]:
torch.save(model.state_dict(), "CalcRankWithGAN")


In [10]:
link1 = "C:\\GIT_BLOG_IMAGE\\"
link2 = "https://github.com/myunghakLee/GIT_BLOG_IMAGE/blob/master/"


In [11]:
STR_A = "C:\GIT_BLOG_IMAGE\clip_image002.png)"
STR_A.replace(link1, link2)

'https://github.com/myunghakLee/GIT_BLOG_IMAGE/blob/master/clip_image002.png)'

In [ ]:
model

In [ ]:
softmax = nn.Softmax(dim=0)

torch.randn((5,1,1))
A = torch.tensor([[[0]], [[1]], [[0]]]).float()
softmax(A)


In [ ]:
A = torch.tensor([[[0],[1],[0]]]).float()
B = torch.tensor([[0],[0.005],[0]]).unsqueeze(0).float()
BCE(A, B)


In [ ]:
softmax = nn.Softmax(dim=-1)
A = torch.tensor([0,0,0,0,0,0,0,0,0]).float()
softmax(A)

In [ ]:
GT

In [ ]:
predictTE

In [ ]:
batch["input"].shape